In [2]:

import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np

# Lemmatize edilmiş CSV dosyasını yükle
df = pd.read_csv('./data_without_extra_features/lemmatized_and_misspelled_removed_SEFACED.csv')

# Veriyi özellikler (lemmatize edilmiş metin) ve hedef değişken olarak ayır
X = df['lemmatized_tokens']
y = df['Class_Label']

# Etiketleri sayısal değerlere dönüştür
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Metinleri tokenle
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)

# Maksimum dizi uzunluğunu tanımla
maxlen = 100

# Dizileri aynı uzunlukta olacak şekilde doldur
X_pad = pad_sequences(X_seq, maxlen=maxlen)

# Veriyi eğitim ve test setlerine ayır
X_train, X_test, y_train, y_test = train_test_split(X_pad, y_encoded, test_size=0.2, random_state=42)

# CNN modelini tanımla
embedding_dim = 100
num_filters = 128
kernel_size = 5

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=maxlen))
model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Modeli derle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Modeli eğit
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Modeli değerlendir
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Kaybı: {loss}, Test Doğruluğu: {accuracy}')


# Eğitilmiş modeli kaydet
y_pred_prob = model.predict(X_test)
y_pred = np.where(y_pred_prob > 0.5, 1, 0)

print(classification_report(y_test, y_pred))


Epoch 1/10
200/200 [==============================] - 52s 200ms/step - loss: 0.2810 - accuracy: 0.8805 - val_loss: 0.0940 - val_accuracy: 0.9594
Epoch 2/10
200/200 [==============================] - 23s 116ms/step - loss: 0.0554 - accuracy: 0.9772 - val_loss: 0.0831 - val_accuracy: 0.9675
Epoch 3/10
200/200 [==============================] - 15s 74ms/step - loss: 0.0320 - accuracy: 0.9869 - val_loss: 0.0860 - val_accuracy: 0.9700
Epoch 4/10
200/200 [==============================] - 10s 52ms/step - loss: 0.0274 - accuracy: 0.9886 - val_loss: 0.0961 - val_accuracy: 0.9675
Epoch 5/10
200/200 [==============================] - 13s 64ms/step - loss: 0.0251 - accuracy: 0.9887 - val_loss: 0.1017 - val_accuracy: 0.9675
Epoch 6/10
200/200 [==============================] - 10s 47ms/step - loss: 0.0244 - accuracy: 0.9887 - val_loss: 0.1079 - val_accuracy: 0.9694
Epoch 7/10
200/200 [==============================] - 10s 49ms/step - loss: 0.0238 - accuracy: 0.9887 - val_loss: 0.1153 - val_accurac